In [3]:
import pandas as pd

In [9]:
hdi_df = pd.read_csv('../data-sources/Human Development Index - Full.csv')
imf_gdp_df = pd.read_csv('../data-sources/imf-gdp.csv')

In [35]:
column_list = [col for col in hdi_df.columns if col.startswith('Human Development Index (')]
column_list = ['Country'] + column_list

hdi_clean_df = hdi_df[column_list].dropna()
clean_df_columns = hdi_clean_df.columns
clean_df_columns = [col.replace(')', '').replace('Human Development Index (', '') for col in clean_df_columns]
hdi_clean_df.columns = clean_df_columns
#hdi_clean_df{str(year) for year in range(2015, 2025)}

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Afghanistan,0.273,0.279,0.287,0.297,0.292,0.310,0.319,0.323,0.324,...,0.466,0.474,0.479,0.478,0.481,0.482,0.483,0.488,0.483,0.478
2,Albania,0.647,0.629,0.614,0.617,0.624,0.634,0.645,0.642,0.657,...,0.778,0.785,0.792,0.795,0.798,0.802,0.806,0.810,0.794,0.796
4,United Arab Emirates,0.728,0.739,0.742,0.748,0.755,0.762,0.767,0.773,0.779,...,0.846,0.852,0.859,0.865,0.870,0.897,0.909,0.920,0.912,0.911
5,Argentina,0.723,0.730,0.735,0.739,0.744,0.745,0.751,0.756,0.762,...,0.843,0.845,0.846,0.848,0.847,0.851,0.850,0.852,0.840,0.842
6,Armenia,0.656,0.649,0.618,0.616,0.620,0.627,0.632,0.641,0.654,...,0.755,0.760,0.764,0.766,0.765,0.768,0.771,0.778,0.757,0.759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,Viet Nam,0.482,0.493,0.504,0.515,0.526,0.539,0.550,0.560,0.569,...,0.672,0.676,0.680,0.684,0.688,0.692,0.697,0.703,0.710,0.703
191,Yemen,0.383,0.386,0.391,0.397,0.399,0.409,0.419,0.427,0.435,...,0.512,0.513,0.505,0.477,0.467,0.459,0.459,0.461,0.460,0.455
192,South Africa,0.632,0.641,0.647,0.652,0.658,0.661,0.656,0.651,0.644,...,0.696,0.704,0.712,0.716,0.719,0.720,0.726,0.736,0.727,0.713
193,Zambia,0.412,0.408,0.408,0.414,0.411,0.413,0.414,0.413,0.411,...,0.548,0.554,0.557,0.562,0.564,0.568,0.572,0.575,0.570,0.565
